# Create EDD Study Files From Data

This notebook creates the files needed for importing a study into Experiment Data Depot (EDD).

## Inputs and outputs

### Required file to run this notebook:
   - `../data/flaviolin/DBTL1/media_descriptions.csv` - media designs for each of the wells
   
   - `../data/flaviolin/DBTL1/OD.xlsx` - production data from the plate reader


### File generated by running this notebook:
   - `edd_experiment_description.csv`
   
   - `edd_protocol.csv`
 
    
The files are stored in the user defined directory.

## Setup

Importing needed libraries:

In [1]:
import sys
sys.path.append('../')

import pandas as pd
import openpyxl

from core import create_media_description

### User parameters

In [2]:
CYCLE = 5

user_params = {
    'media_file': f'../data/flaviolin/DBTL{CYCLE}/media_descriptions.csv',  
    'measurement_file': f'../data/flaviolin/DBTL{CYCLE}/OD.xlsx',
    'output_file_path': f'../data/flaviolin/DBTL{CYCLE}', # Folder for output files,
    'num_replicates': 3,
    'num_designs': 16,
    'protocol_name': ['OD600', 'OD340'],
    'time_point': 48,
    'part_id': 'JBx_193086',
    'media': 'MOPS',
    'culture_volume': 15,
    'well_volume': 1500,
    'shaking_speed': 800,
    'temperature': 30,
    } 


In [3]:
df = pd.read_csv(user_params['media_file'], index_col=0)
df.head()

,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4
Well,,,,,,,,,,,,,,,
A1,40.0,4.0,0.030851,20.0,0.193823,2.968560,0.062151,13.770459,2.158244,327.434888,0.000126,0.002049,0.000767,0.000625,0.000542
B1,40.0,4.0,0.030851,20.0,0.193823,2.968560,0.062151,13.770459,2.158244,327.434888,0.000126,0.002049,0.000767,0.000625,0.000542
C1,40.0,4.0,0.030851,20.0,0.193823,2.968560,0.062151,13.770459,2.158244,327.434888,0.000126,0.002049,0.000767,0.000625,0.000542
D1,40.0,4.0,0.026618,20.0,0.178729,5.271786,0.046587,10.728408,2.566785,338.143039,0.000106,0.001254,0.000751,0.002341,0.000775
E1,40.0,4.0,0.026618,20.0,0.178729,5.271786,0.046587,10.728408,2.566785,338.143039,0.000106,0.001254,0.000751,0.002341,0.000775


## Create Line Description

In [4]:
df['Line Description'] = df.apply(create_media_description, axis=1)


## Create Line Names

Add metadata for media and replicates to craft Line Names as **C[.]\_W[.]1\_[.]3-R[.]** denoting cycle number, wells occupying the same design and replicate number:

Check if it's row or column order of lines:

In [5]:
column_order = True if df.index[1][0] == 'B' else False

In [6]:
reps = user_params['num_replicates']
num_media_designs = user_params['num_designs']

def linefunction_colum_order(row):
    well = row.name
    if well[0] in 'ABC':
        return f'C{CYCLE}_WA{well[1]}_C{well[1]}-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_WD{well[1]}_F{well[1]}-R{row["Replicate"]}'                                                                                              

def linefunction_row_order(row):
    well = row.name
    if int(well[1]) < 5:
        return f'C{CYCLE}_W{well[0]}1_{well[0]}4-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_W{well[0]}5_{well[0]}8-R{row["Replicate"]}'                                                                                              

    
df['Replicate'] = [i+1 for _ in range(num_media_designs) for i in range(reps)]

if column_order:
    df['Line Name'] = df.apply(linefunction_colum_order, axis=1)
else:
    df['Line Name'] = df.apply(linefunction_row_order, axis=1)

In [7]:
df.head(2)

,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4,Line Description,Replicate,Line Name
Well,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.030851,20.0,0.193823,2.96856,0.062151,13.770459,2.158244,327.434888,0.000126,0.002049,0.000767,0.000625,0.000542,"MOPS: 40.000000, Tricine: 4.000000, H3BO3: 0.0...",1,C5_WA1_C1-R1
B1,40.0,4.0,0.030851,20.0,0.193823,2.96856,0.062151,13.770459,2.158244,327.434888,0.000126,0.002049,0.000767,0.000625,0.000542,"MOPS: 40.000000, Tricine: 4.000000, H3BO3: 0.0...",2,C5_WA1_C1-R2


## Process measurement files

Read measurements file:

In [8]:
df_600 = pd.read_excel(user_params['measurement_file'], sheet_name='600', index_col=0)
df_340 = pd.read_excel(user_params['measurement_file'], sheet_name='340', index_col=0)

df_600

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.1669,0.1287,0.1221,0.1599,0.1428,0.1593,0.1203,0.1449,0.0358,0.0464,0.0461,0.0459
B,0.1405,0.1267,0.1290,0.1347,0.1563,0.1442,0.1384,0.1459,0.0469,0.0466,0.0465,0.0457
C,0.1514,0.1527,0.1357,0.1349,0.1985,0.1499,0.1324,0.1457,0.0465,0.0467,0.0466,0.0456
D,0.1898,0.1532,0.1514,0.1423,0.1499,0.1268,0.1369,0.1827,0.0466,0.0465,0.0464,0.0457
E,0.1205,0.1290,0.1367,0.1113,0.1605,0.1182,0.1066,0.1641,0.0465,0.0464,0.0491,0.0460
F,0.1141,0.1174,0.1255,0.1072,0.1460,0.1217,0.0803,0.1221,0.0467,0.0464,0.0461,0.0457
G,0.0461,0.0464,0.0468,0.0466,0.0467,0.0462,0.0462,0.0465,0.0466,0.0464,0.0462,0.0458
H,0.0461,0.0461,0.0460,0.0461,0.0456,0.0457,0.0458,0.0457,0.0461,0.0461,0.0459,0.0457


In [9]:
df_340

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.5700,0.6891,0.7094,0.5925,0.6770,0.7258,0.7460,0.6288,0.1228,0.1351,0.1333,0.1313
B,0.5761,0.6952,0.7099,0.7097,0.7575,0.7192,0.7482,0.6305,0.1389,0.1377,0.1348,0.1308
C,0.6146,0.6738,0.6432,0.6876,0.7515,0.7376,0.7388,0.6043,0.1392,0.1392,0.1361,0.1312
D,0.7128,0.7459,0.6805,0.7498,0.5255,0.7438,0.7521,0.3704,0.1390,0.1386,0.1363,0.1333
E,0.6561,0.7308,0.6735,0.7752,0.5931,0.7436,0.7319,0.3726,0.1385,0.1382,0.1365,0.1322
F,0.6999,0.7308,0.7241,0.7391,0.5827,0.7182,0.7531,0.3544,0.1388,0.1389,0.1368,0.1327
G,0.1318,0.1343,0.1380,0.1391,0.1391,0.1376,0.1376,0.1395,0.1394,0.1382,0.1348,0.1321
H,0.1319,0.1340,0.1354,0.1351,0.1356,0.1350,0.1349,0.1372,0.1351,0.1348,0.1331,0.1318


### Process OD600

Normalize the data to the control well A9 value (water content):

In [10]:
zero_value_600 = df_600.at['A', 9]
df_600.loc[:, df_600.columns] -= zero_value_600
df_600

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.1311,0.0929,0.0863,0.1241,0.1070,0.1235,0.0845,0.1091,0.0000,0.0106,0.0103,0.0101
B,0.1047,0.0909,0.0932,0.0989,0.1205,0.1084,0.1026,0.1101,0.0111,0.0108,0.0107,0.0099
C,0.1156,0.1169,0.0999,0.0991,0.1627,0.1141,0.0966,0.1099,0.0107,0.0109,0.0108,0.0098
D,0.1540,0.1174,0.1156,0.1065,0.1141,0.0910,0.1011,0.1469,0.0108,0.0107,0.0106,0.0099
E,0.0847,0.0932,0.1009,0.0755,0.1247,0.0824,0.0708,0.1283,0.0107,0.0106,0.0133,0.0102
F,0.0783,0.0816,0.0897,0.0714,0.1102,0.0859,0.0445,0.0863,0.0109,0.0106,0.0103,0.0099
G,0.0103,0.0106,0.0110,0.0108,0.0109,0.0104,0.0104,0.0107,0.0108,0.0106,0.0104,0.0100
H,0.0103,0.0103,0.0102,0.0103,0.0098,0.0099,0.0100,0.0099,0.0103,0.0103,0.0101,0.0099


Set negative values to zero, multiply the values by 10 to account for 10x dilution and keep only 8 columns and 6 rows:

In [11]:
df_600[df_600 < 0] = 0
df_600.loc[:, df_600.columns] *= 10
df_600 = df_600.iloc[:6,:8]
df_600

,1,2,3,4,5,6,7,8
A,1.311,0.929,0.863,1.241,1.070,1.235,0.845,1.091
B,1.047,0.909,0.932,0.989,1.205,1.084,1.026,1.101
C,1.156,1.169,0.999,0.991,1.627,1.141,0.966,1.099
D,1.540,1.174,1.156,1.065,1.141,0.910,1.011,1.469
E,0.847,0.932,1.009,0.755,1.247,0.824,0.708,1.283
F,0.783,0.816,0.897,0.714,1.102,0.859,0.445,0.863


### Process OD340

Do the same for OD340, except of multiplication by 10, as the OD340 measurements were taken from non-diluted samples:

In [12]:
zero_value_340 = df_340.at['A', 9]
df_340.loc[:, df_340.columns] -= zero_value_340
df_340[df_340 < 0] = 0
df_340 = df_340.iloc[:6,:8]
df_340

,1,2,3,4,5,6,7,8
A,0.4472,0.5663,0.5866,0.4697,0.5542,0.6030,0.6232,0.5060
B,0.4533,0.5724,0.5871,0.5869,0.6347,0.5964,0.6254,0.5077
C,0.4918,0.5510,0.5204,0.5648,0.6287,0.6148,0.6160,0.4815
D,0.5900,0.6231,0.5577,0.6270,0.4027,0.6210,0.6293,0.2476
E,0.5333,0.6080,0.5507,0.6524,0.4703,0.6208,0.6091,0.2498
F,0.5771,0.6080,0.6013,0.6163,0.4599,0.5954,0.6303,0.2316


Transform wide to long format, matching the order of wells in index to the one from `df`:

In [13]:
if column_order:
    indfcn = lambda x: f'{x.name}{int(x["variable"])}'
    df_600 = df_600.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.melt(ignore_index=False, value_name='OD340')
else:
    indfcn = lambda x: f'{x["variable"]}{(x.name)}'
    df_600 = df_600.T.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.T.melt(ignore_index=False, value_name='OD340')

df_600['Well'] = df_600.apply(indfcn, axis=1)
df_340['Well'] = df_340.apply(indfcn, axis=1)

df_600.index = df_600['Well']
df_340.index = df_340['Well']

df_600.drop(columns=['variable', 'Well'], inplace=True)
df_340.drop(columns=['variable', 'Well'], inplace=True)

df_600.head()

,OD600
Well,
A1,1.311
B1,1.047
C1,1.156
D1,1.540
E1,0.847


Add measurements to the main dataframe:

In [14]:
measurOD600 = user_params['protocol_name'][0]
measurOD340 = user_params['protocol_name'][1]
df[measurOD600] = df_600
df[measurOD340] = df_340
df.head(2)


,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4,Line Description,Replicate,Line Name,OD600,OD340
Well,,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.030851,20.0,0.193823,2.96856,0.062151,13.770459,2.158244,327.434888,0.000126,0.002049,0.000767,0.000625,0.000542,"MOPS: 40.000000, Tricine: 4.000000, H3BO3: 0.0...",1,C5_WA1_C1-R1,1.311,0.4472
B1,40.0,4.0,0.030851,20.0,0.193823,2.96856,0.062151,13.770459,2.158244,327.434888,0.000126,0.002049,0.000767,0.000625,0.000542,"MOPS: 40.000000, Tricine: 4.000000, H3BO3: 0.0...",2,C5_WA1_C1-R2,1.047,0.4533


## Create EDD Experiment Description File

In [15]:
df['Media'] = user_params['media']
df['Part ID'] = user_params['part_id']
df['Culture Volume'] = user_params['culture_volume']
df['Flask Volume'] = user_params['well_volume']
df['Growth Temperature'] = user_params['temperature']
df['Shaking speed'] = user_params['shaking_speed']
# df['Starting OD'] =
# df['Replicate Count'] = 24


In [16]:
# Invalid columns for now in EDD
# df['Humidity[%]'] = user_params['humidity']
# df['Plate'] = user_params['plate']

In [17]:
exp_descr_file = f'{user_params["output_file_path"]}/edd_experiment_description.xlsx'
df[['Line Name',
    'Line Description',
    'Part ID',
    'Media',
    'Culture Volume',
    'Flask Volume',
    'Growth Temperature',
    'Shaking speed',
]].to_excel(exp_descr_file, index=False)

## Create EDD Measurement File

OD600

In [18]:
measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD600}.xlsx'
df['Measurement Type'] = 'Optical Density'

df['Time'] = user_params['time_point']
df['Value'] = df[measurOD600]
df['Units'] = 'n/a'
df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

OD340

In [19]:
measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD340}.xlsx'

df['Time'] = user_params['time_point']
df['Value'] = df[measurOD340]
df['Units'] = 'n/a'
df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)